In [1]:
from sklearn.model_selection import train_test_split
from sentence_transformers import InputExample

import pandas as pd
import pickle

In [16]:
reports_file_path = 'data/br/eclipse/eclipse.csv'
relations_file_path = 'data/br/eclipse/eclipse_pairs.csv'

reports_eclipse = pd.read_csv(reports_file_path, index_col='bug_id')
relations_eclipse = pd.read_csv(relations_file_path, index_col='issue_id')

In [3]:
reports_file_path = 'data/br/openoffice/openoffice.csv'
relations_file_path = 'data/br/openoffice/openoffice_pairs.csv'

reports_openoffice = pd.read_csv(reports_file_path, index_col='bug_id')
relations_openoffice = pd.read_csv(relations_file_path, index_col='issue_id')

In [8]:
reports_file_path = 'data/br/firefox/firefox.csv'
relations_file_path = 'data/br/firefox/firefox_pairs.csv'

reports_firefox = pd.read_csv(reports_file_path, index_col='bug_id')
relations_firefox = pd.read_csv(relations_file_path, index_col='issue_id')

In [5]:
reports_file_path = './data/br/netbeans/netbeans.csv'
relations_file_path = 'data/br/netbeans/netbeans_pairs - Copia.csv'

reports_netbeans = pd.read_csv(reports_file_path, index_col='bug_id')
relations_netbeans = pd.read_csv(relations_file_path, index_col='issue_id')

In [6]:
datasets = {}

In [7]:
datasets['eclipse'] = {
    'reports': reports_eclipse,
    'relations': relations_eclipse
}

datasets['openoffice'] = {
    'reports': reports_openoffice,
    'relations': relations_openoffice
}

datasets['firefox'] = {
    'reports': reports_firefox,
    'relations': relations_firefox
}

datasets['netbeans'] = {
    'reports': reports_netbeans,
    'relations': relations_netbeans
}

In [8]:
def generate_triplets(reports, relations):
    
    train_examples = []
    for index, value in reports.iterrows():
        if index in relations.index:
            duplicates_id = []
            try:
                duplicates_id = [int(id) for id in relations.loc[index].values[0].split(';')]
            except:
                duplicates_id = []

            for id in duplicates_id:
                if id in reports.index:
                    
                    positive = reports.loc[id]
                    negative = reports.sample(n=1).iloc[0]
                    while (negative.name in duplicates_id) and not isinstance(negative['description'], str):
                        negative = reports.sample(n=1).iloc[0]
                        print(negative)

                    if isinstance(value['description'], str) and isinstance(positive['description'], str):
                        train_examples.append(
                            InputExample(texts=[
                                value['description'],
                                positive['description'],
                                negative['description']
                            ])
                        )

                    else:
                        print('save')
                        
    return train_examples

In [9]:
for key, value in datasets.items():

    # Split data into train, validation and test sets
    reports_temp, reports_test = train_test_split(value['reports'], test_size=0.3, random_state=83)
    reports_train, reports_val = train_test_split(reports_temp, test_size=0.3, random_state=83)

    print(len(value['relations']))
    # Save each split into a csv file
    reports_train.to_csv(f'data/splits/{key}/{key}_train.csv')
    reports_val.to_csv(f'data/splits/{key}/{key}_val.csv')
    reports_test.to_csv(f'data/splits/{key}/{key}_test.csv')

    # Generate triplets for the train dataset
    triplets = generate_triplets(reports_train, value['relations'])
    print(key, len(triplets))

    # Save triplets into pickle file
    triplets_file_path = f'data/splits/{key}/{key}_triplets.pkl'
    with open(triplets_file_path, "wb") as f:
        pickle.dump(triplets, f)

    

193599
eclipse 20915
54402
openoffice 14453
92651
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
firefox 74436
118960
netbeans 22777


In [12]:
train_triplets = {}

for key, value in datasets.items():
    
    triplets_file_path = f'data/splits/{key}/{key}_triplets.pkl'

    with open(triplets_file_path, "rb") as f:
        loaded_input_examples = pickle.load(f)

    train_triplets[key] = loaded_input_examples

    print(key, len(loaded_input_examples))

eclipse 20915
openoffice 14453
firefox 74436
netbeans 22777


In [14]:
type(train_triplets['firefox'][0])

sentence_transformers.readers.InputExample.InputExample

In [16]:
all_triplets = []

for list_of_triplets in train_triplets.values():
    all_triplets.extend(list_of_triplets)

len(all_triplets)

132581

In [18]:
reports_train_file_path = 'data/splits/eclipse/eclipse_test.csv'


reports_eclipse_test = pd.read_csv(reports_train_file_path, index_col='bug_id')

In [19]:
len(reports_eclipse_test)

108302

In [23]:
def count_duplictes(reports, relations):

    amount_of_duplicates = {}
    for index, value in reports.iterrows():
        duplicates_id = []
        if index in relations.index:
            try:
                duplicates_id = [int(id) for id in relations.loc[index].values[0].split(';')]
            except:
                duplicates_id = []

        
        if len(duplicates_id) > 0:
            amount_of_duplicates[index] = 0

            for id in duplicates_id:
                if id in reports.index:
                    positive = reports.loc[id]
                    if isinstance(value['description'], str) and isinstance(positive['description'], str):
                        amount_of_duplicates[index] +=1
                        
    return amount_of_duplicates

In [27]:
test = count_duplictes(reports_eclipse_test, relations_eclipse)

In [28]:
len(test)

11892

In [29]:
sum(test.values())/len(test)

0.6732256979482004

In [30]:
len(reports_eclipse_test)

108302